In [8]:
from flopth import flopth
import torch
import pandas as pd
import torchvision
import os
import torch.nn as nn
import segmentation_models_pytorch as smp



In [15]:
def get_flops(model):
    dummy_inputs = torch.rand(1, 1, 256, 256)
    flops, params = flopth(model, inputs=(dummy_inputs,))
    return flops, params

In [16]:
csv_dir = "/home/anees.hashmi/Desktop/labs/hc701_assignment_3/csv_liver_only"
model_dir = "/home/anees.hashmi/Desktop/labs/hc701_assignment_3/models"
device = "cuda"
num_workers = 24
batch_size = 1
num_classes = 2
model_path ="/home/anees.hashmi/Desktop/labs/hc701_assignment_3/models/"
result_dir ="./results/"
backbone = "mobilenet_v2"
model_name = "unet_dice_loss_mobilenet_v2_1.0_1.0"


all_model_names = ['unet_dice_loss_mobilenet_v2_1.0_1.0', 'unet_dice_loss_mobilenet_v2_1.0_5.0', 
                   'unet_BCE_loss_mobilenet_v2_1.0_5.0', 'unet_dice_loss_resnet101_1.0_5.0']
backbones= {'mobilenet':"mobilenet_v2", "resnet101":"resnet101"}

checkpoint = None

if not checkpoint:
    all_chckpoints = [i[:-4] for i in sorted(os.listdir(f"{model_path}{model_name}"))]
    checkpoint = f"""{["_".join(i) for i in sorted([i.split('_') for i in all_chckpoints], key=lambda x: int(x[-4]))][-1]}.pth"""


In [17]:
def load_model(backbone, device, model_dir, model_name, checkpoint, num_classes):
    model = smp.Unet(
        encoder_name=backbone,        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
        # encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=num_classes,                      # model output channels (number of classes in your dataset)
    )
    model = model.to(device)
    model.load_state_dict(torch.load(os.path.join(model_dir, model_name, checkpoint)))
    return model

sig = nn.Sigmoid()

model = load_model(backbone, device, model_dir, model_name, checkpoint, num_classes)

In [18]:
for model_name in all_model_names:
    backbone = backbones[model_name.split("_")[3]]
    all_chckpoints = [i[:-4] for i in sorted(os.listdir(f"{model_path}{model_name}"))]
    checkpoint = f"""{["_".join(i) for i in sorted([i.split('_') for i in all_chckpoints], key=lambda x: int(x[-4]))][-1]}.pth"""
    model = load_model(backbone=backbone, device=device, model_dir= model_dir, model_name=model_name, checkpoint = checkpoint, num_classes=num_classes)
    flops, params = get_flops(model)
    print(f"{model_name} : {flops} flops, {params} params")


unet_dice_loss_mobilenet_v2_1.0_1.0 : 3.38276G flops, 6.62851M params
unet_dice_loss_mobilenet_v2_1.0_5.0 : 3.38276G flops, 6.62851M params
unet_BCE_loss_mobilenet_v2_1.0_5.0 : 3.38276G flops, 6.62851M params
unet_dice_loss_resnet101_1.0_5.0 : 15.4563G flops, 51.5071M params
